### Temperature Model

####  Model Equation 
$R =R_{max}*\left(\frac{T_{max}-Temp}{T_{max}-T_{opt}}\right)*\left(\frac{Temp-T_{min}}{T_{opt}-T_{min}}\right)^\frac{T_{opt}-T_{min}}{T_{max}-T_{opt}}$

#### Parameters
$R$: Growth Rate  
$R_{max}$: Maximal Growth Rate (4.0)  
$T_{max}$: Maximal Growth Temperature (30)  
$T_{min}$: Minimal Growth Temperature (-5)       
$T_{opt}$: Optimal Growth Temperature (17.6)  

#### Required Input
Monthly maximal temperature

### Stream Flow Model

#### Chinook
Spawn_Area = a/(1+((Flow-Fo)/b)^2)
Fo = 424  
a = 12818  
b = 424  

#### Sockeye
Spawn_Area = a/(1+((Flow-Fo)/b)^2)  
Fo = 390  
a = 946   
b = 390  

#### Coho
Spawn_Area = a/(1+((Flow-Fo)/b)^2)  
Fo = 362   
a = 12124   
b = 362  

#### Pink
Spawn_Area = Amax*((Fmax-Flow)/(Fmax-Fopt))*((Flow-Fmin)/(Fopt-Fmin))^((Fopt-Fmin)/(Fmax-Fopt))  
Amax = 1052  
Fmax = 1200   
Fmin = 0   
Fopt = 420  

### Model Run and Output Interpretation

#### Reading in future projections of stream temperature and stream flow

In [1]:
import fishmod_utils
import pandas as pd

In [2]:
data = pd.read_csv("sites_streamflow_stream_temperature.csv")
data.head()

,Site ID,Stream Temperature 2040s,Stream Temperature 2080s,Streamflow 2040s,Streamflow 2080s
0,ALB,24.304,25.621,2042.155229,1955.022943
1,LIB,18.852,19.938,2727.771443,2488.091671
2,BFE,12.079,12.824,4217.675486,3687.116386
3,BON,19.896,21.094,67905.510514,70655.700086
4,HOD,22.852,23.982,251.917157,250.765671


#### Feeding futrue stream temperature into temperature model and translating outputs into fish happiness scale 1-5

In [24]:
# Stream temperature 2040s
temp = data["Stream Temperature 2040s"]
temp_vulnerability40 = fishmod_utils.temp_mod(temp, r_max=4, t_max=30, t_min=-5, t_opt=17.6)

model output = temp_vulnerability


In [25]:
# Stream temperature 2080s
temp = data["Stream Temperature 2080s"]
temp_vulnerability80 = fishmod_utils.temp_mod(temp, r_max=4, t_max=30, t_min=-5, t_opt=17.6)

model output = temp_vulnerability


#### Feeding futrue streamflow into streamflow model and translating outputs into fish happiness scale 1-5

In [26]:
## Stream flow 2040s
flow = data["Streamflow 2040s"]

# Chinook
flow_vulnerability40_chinook = fishmod_utils.stream_mod(flow, fo=424, a=12818, b=424)
# Sockeye
flow_vulnerability40_sockeye = fishmod_utils.stream_mod(flow, fo=390, a=946, b=390)
# Coho
flow_vulnerability40_coho = fishmod_utils.stream_mod(flow, fo=362, a=12124, b=362)
# Pink
flow_vulnerability40_pink = fishmod_utils.stream_mod_pink(flow, a_max=1052, f_max=1200, f_min=0, f_opt=420)

model output = flow_vulnerability
model output = flow_vulnerability
model output = flow_vulnerability
model output = flow_vulnerability_pink


In [27]:
## stream flow 2080s
flow = data["Streamflow 2080s"]

# Chinook
flow_vulnerability80_chinook = fishmod_utils.stream_mod(flow, fo=424, a=12818, b=424)
# Sockeye
flow_vulnerability80_sockeye = fishmod_utils.stream_mod(flow, fo=390, a=946, b=390)
# Coho
flow_vulnerability80_coho = fishmod_utils.stream_mod(flow, fo=362, a=12124, b=362)
# Pink
flow_vulnerability80_pink = fishmod_utils.stream_mod_pink(flow, a_max=1052, f_max=1200, f_min=0, f_opt=420)

model output = flow_vulnerability
model output = flow_vulnerability
model output = flow_vulnerability
model output = flow_vulnerability_pink


#### Combining temperature & streamflow vulnerability projections, assigning equal weight to each output

In [28]:
## Year 2040s
# Chinook
chinook_vulnerability40 = pd.DataFrame(temp_vulnerability40*0.5 + flow_vulnerability40_chinook*0.5)
# Sockeye
sockeye_vulnerability40 = pd.DataFrame(temp_vulnerability40*0.5 + flow_vulnerability40_sockeye*0.5)
# Coho
coho_vulnerability40 = pd.DataFrame(temp_vulnerability40*0.5 + flow_vulnerability40_coho*0.5)
# Pink
pink_vulnerability40 = pd.DataFrame(temp_vulnerability40*0.5 + flow_vulnerability40_pink*0.5)

In [31]:
## Year 2080s
# Chinook
chinook_vulnerability80 = pd.DataFrame(temp_vulnerability80*0.5 + flow_vulnerability80_chinook*0.5)
# Sockeye
sockeye_vulnerability80 = pd.DataFrame(temp_vulnerability80*0.5 + flow_vulnerability80_sockeye*0.5)
# Coho
coho_vulnerability80 = pd.DataFrame(temp_vulnerability80*0.5 + flow_vulnerability80_coho*0.5)
# Pink
pink_vulnerability80 = pd.DataFrame(temp_vulnerability80*0.5 + flow_vulnerability80_pink*0.5)

#### Combining all model outputs to a dataframe

In [51]:
Site = data["Site ID"]
fish_vulnerability = pd.concat([Site, chinook_vulnerability40, sockeye_vulnerability40, coho_vulnerability40, pink_vulnerability40,
                                chinook_vulnerability80, sockeye_vulnerability80, coho_vulnerability80, pink_vulnerability80], axis=1)
fish_vulnerability.columns = ["Site_ID","Chinook40", "Sockeye40", "Coho40", "Pink40","Chinook80", "Sockeye80", "Coho80", "Pink80"]

In [53]:
fish_vulnerability.head()

,Site_ID,Chinook40,Sockeye40,Coho40,Pink40,Chinook80,Sockeye80,Coho80,Pink80
0,ALB,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1,LIB,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,BFE,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,BON,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
4,HOD,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
5,JDA,1.5,1.5,1.5,1.5,1.0,1.0,1.0,1.0
6,KLC,5.0,5.0,5.0,5.0,5.0,5.0,4.5,5.0
7,MCN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,PEL,5.0,5.0,5.0,5.0,5.0,5.0,4.5,5.0
9,ROU,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
